# [LVV-T235] - M13T-012: Position Repeatability After Parking

Notebook containing data analysis for the [LVV-T235] test case.  
The script used to run this test case can be found in [lsst-ts/ts_m1m3supporttesting/M13T012.py].  

[LVV-T235]: https://jira.lsstcorp.org/secure/Tests.jspa#/testCase/LVV-T235
[lsst-ts/ts_m1m3supporttesting/M13T012.py]: https://github.com/lsst-ts/ts_m1m3supporttesting/blob/develop/M13T012.py

## Prepare Notebook

In [ ]:
test_case = "LVV-T235"

test_exec = "LVV-E985"
t_start = "2023-03-07T18:48:20" # Exact time clean up initial fault
t_end = "2023-03-07T19:30:00"

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import itertools as itt
import pandas as pd
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from astropy import units as u
from astropy.time import Time, TimezoneInfo

from lsst.sitcom import vandv
from lsst.ts.idl.enums import MTM1M3

In [ ]:
client = vandv.efd.create_efd_client()

## Collect Data

In [ ]:
all_columns = ["xPosition", "xRotation", "yPosition", "yRotation", "zPosition", "zRotation"]
pos_columns = [c for c in all_columns if "Position" in c]
rot_columns = [c for c in all_columns if "Rotation" in c]
print(pos_columns, rot_columns)

In [ ]:
df_ims = await client.select_time_series(
    "lsst.sal.MTM1M3.imsData", 
    "*", 
    Time(t_start, format="isot", scale="utc"),
    Time(t_end, format="isot", scale="utc"), 
)

df_ims = df_ims.set_index("private_rcvStamp")
df_ims.index = pd.to_datetime(df_ims.index, unit="s")
df_ims = df_ims[all_columns]

# Convert meter to milimeter to make is easier to analyse
df_ims[pos_columns] = df_ims[pos_columns] * 1e3

# Convert radians to degrees
df_ims[rot_columns] = np.rad2deg(df_ims[rot_columns])

In [ ]:
df_state = await client.select_time_series(
    "lsst.sal.MTM1M3.logevent_detailedState", 
    "*", 
    Time(t_start, format="isot", scale="utc"),
    Time(t_end, format="isot", scale="utc"), 
)

df_state["detailedStateName"] = \
    df_state["detailedState"].map(lambda x: MTM1M3.DetailedState(x).name)

df_state = df_state.set_index("private_rcvStamp")
df_state.index = pd.to_datetime(df_state.index, unit="s")

In [ ]:
df_cmdPos = await client.select_time_series(
    "lsst.sal.MTM1M3.command_positionM1M3", 
    "*", 
    Time(t_start, format="isot", scale="utc"),
    Time(t_end, format="isot", scale="utc"), 
)

df_cmdPos = df_cmdPos.set_index("private_rcvStamp")
df_cmdPos.index = pd.to_datetime(df_cmdPos.index, unit="s")
df_cmdPos = df_cmdPos[all_columns]
df_cmdPos = df_cmdPos * 1e3 # Convert meter to milimeter to make is easier to analyse

In [ ]:
df_HPState = await client.select_time_series(
    "lsst.sal.MTM1M3.logevent_hardpointActuatorState", 
    "*", 
    Time(t_start, format="isot", scale="utc"),
    Time(t_end, format="isot", scale="utc"), 
)

df_HPState = df_HPState.set_index("private_rcvStamp")
df_HPState.index = pd.to_datetime(df_HPState.index, unit="s")

## Initial Data Display 

The plot below shows the whole data collection.  
There you can see a loop repeated seven times. 

In [ ]:
title = f"{test_case} {test_exec}\nData Overview"
fig, axs = plt.subplots(num=title, nrows=3, sharex=True, figsize=(10, 5))

when_parked = df_state[df_state["detailedStateName"] == "PARKED"].index
when_rasing = df_state[df_state["detailedStateName"] == "RAISINGENGINEERING"].index
when_lowering = df_state[df_state["detailedStateName"] == "LOWERINGENGINEERING"].index
when_active = df_state[df_state["detailedStateName"] == "ACTIVEENGINEERING"].index

for i, label in enumerate(pos_columns):
    ax = axs[i]
    ax.plot(df_ims[label])
    
    for idx in when_parked:
        l1 = ax.axvline(idx, lw="0.5", c="k")
        
    for idx in when_rasing:
        l2 = ax.axvline(idx, lw="0.5", c="k", ls="--")
        
    for idx in when_lowering:
        l3 = ax.axvline(idx, lw="0.5", c="k", ls=":")
        
    for idx in when_active:
        l4 = ax.axvline(idx, lw="0.5", c="C1", ls="-")

    ax.grid(":", lw=0.1)
    ax.set_xlabel("Time [UTC]")
    ax.set_ylabel(f"{label} [mm]")

fig.legend(
    [l1, l2, l3, l4], 
    ["PARKED", "RAISINGENGINEERING", "LOWERINGENGINEERING", "ACTIVEENGINEERING"], 
    ncols=4, 
    loc="upper right", 
    bbox_to_anchor=(0.93, 0.92)
)
fig.suptitle(title + "\n")
fig.autofmt_xdate()
fig.tight_layout()

plt.show()

### Find and Subtract Reference Value

As you can see, the initial M1M3 position is not zero on any of the directions.  
It will be much easier if we can do the data analysis on data that is centered at zero.  
From the plot above, we can get the telemetry close to when the ACTIVEENGINEERING detailed state is as a reference.
  
The cell bellow shows how we extract the median near the detailed state event associated with ACTIVEENGINEERING.

In [ ]:
when_active = df_state[df_state["detailedStateName"] == "ACTIVEENGINEERING"].index

sub_df = pd.DataFrame(columns=df_ims.columns.to_list())

for idx in when_active:
    dt = pd.Timedelta(2, "sec")
    temp = df_ims.loc[idx:idx+dt]
    sub_df = pd.concat((sub_df, temp), axis=0)
    
median_vals = sub_df.median()
print(median_vals)

  
Now we copy the IMS dataframe and subtract the reference values from all the position columns.  

In [ ]:
df_results = df_ims.copy() 

for col in pos_columns:
    df_results[col] = df_results[col] - median_vals[col]

And plot them.

In [ ]:
title = f"{test_case} {test_exec}\nData Overview with Reference Subtracted"
fig, axs = plt.subplots(num=title, nrows=3, sharex=True, figsize=(10, 5))

when_parked = df_state[df_state["detailedStateName"] == "PARKED"].index

for i, label in enumerate(["xPosition", "yPosition", "zPosition"]):
    ax = axs[i]
    ax.plot(df_results[label])
    
    for idx in when_parked:
        ax.axvline(idx, lw="0.5", c="k")
        
    for idx in when_rasing:
        ax.axvline(idx, lw="0.5", c="k", ls="--")
        
    for idx in when_lowering:
        ax.axvline(idx, lw="0.5", c="k", ls=":")
        
    for idx in when_active:
        ax.axvline(idx, lw="0.5", c="C1", ls="-")

    ax.grid(":", lw=0.1)
    ax.set_xlabel("Time [UTC]")
    ax.set_ylabel(f"{label} [mm]")

fig.legend(
    [l1, l2, l3, l4], 
    ["PARKED", "RAISINGENGINEERING", "LOWERINGENGINEERING", "ACTIVEENGINEERING"], 
    ncols=4, 
    loc="upper right", 
    bbox_to_anchor=(0.93, 0.92)
)
    
fig.suptitle(title + "\n")
fig.autofmt_xdate()
fig.tight_layout()

plt.show()

### Zoom Single Loop

The plots above show the whole data acquisition process.  
They show a loop of seven iterations through the different positions.  
The plot below shows a zoom in the first interation that starts in the first PARK state event and finished in the second PARK state event. 
It also shows the commanded position and the time window we use for statistics. 

In [ ]:
title = f"{test_case} {test_exec}\nZoom Single Loop"
fig, axs = plt.subplots(num=title, nrows=3, sharex=True, figsize=(10, 5))

when_parked = df_state[df_state["detailedStateName"] == "PARKED"].index
axes = ["x", "y", "z"]
colors = ["g", "b", "r"]

for i, label in enumerate(["xPosition", "yPosition", "zPosition"]):
    ax = axs[i]
    sub_df_results = df_results[when_parked[0]:when_parked[1]]
    sub_df_cmdPos = df_cmdPos[when_parked[0]:when_parked[1]]
    
    l1, = ax.plot(sub_df_results[label])
    
    sub_sub_df_cmdPos = sub_df_cmdPos[sub_df_cmdPos[label] != 0]

    for idx in sub_sub_df_cmdPos.index:
                
        sub_df_HPState = df_HPState[idx:]        
        mask = (sub_df_HPState[[f"motionState{j}" for j in range(1, 6)]] == 0).all(axis=1) 
        idx_hp = sub_df_HPState[mask].index[0]

        l4 = ax.axvspan(idx_hp, idx_hp+pd.Timedelta(2, "sec"), fc="firebrick", alpha=0.5)
        l3 = ax.axvline(idx_hp, c="k", ls=":")
        l2 = ax.axvline(idx, c="k", ls="-")

    ax.grid(":", lw=0.2)
    ax.set_xlabel("Time [UTC]")
    ax.set_xlim(sub_df_results.index[0], sub_df_results.index[-1])
    ax.set_ylabel(f"{label} [mm]")

leg = fig.legend(
    [l1, l2, l3, l4], 
    ["Position", "Commanded Position", "Hard-Points in StandBy", "Data Collection Window"], 
    ncols=4,
    loc="upper right",
    bbox_to_anchor=(0.94, 0.91)
)
leg.get_frame().set_linewidth(0.0)

fig.suptitle(title + "\n")
fig.autofmt_xdate()
fig.tight_layout()

plt.show()

## Super Zoom in each commanded position

Here we do a preliminary analysis on each of the commanded X/Y position.  
Yes, we are omitting the 

In [ ]:
dt = pd.Timedelta(2, "seconds")
when_parked = df_state[df_state["detailedStateName"] == "PARKED"].index

title = f"{test_case} {test_exec}\nSuper Zoom for +/- X/Y Positions"
fig, axs = plt.subplots(num=title, nrows=2, ncols=2, figsize=(10, 5), sharex=True)

for loop in range(when_parked.size-1):

    sub_df_results = df_results[when_parked[loop]:when_parked[loop+1]]
    sub_df_cmdPos = df_cmdPos[when_parked[loop]:when_parked[loop+1]]

    for i, label in enumerate(["xPosition", "yPosition"]):
        sub_sub_df_cmdPos = sub_df_cmdPos[sub_df_cmdPos[label] != 0]
        for ii, idx in enumerate(sub_sub_df_cmdPos.index):
            sub_df_HPState = df_HPState[idx:]        
            mask = (sub_df_HPState[[f"motionState{j}" for j in range(1, 6)]] == 0).all(axis=1) 
            idx_hp = sub_df_HPState[mask].index[0]
            
            temp = df_results.loc[idx_hp:idx_hp+dt, label].copy()
            temp.index = temp.index - temp.index[0]

            ax = axs[i, ii]
            ax.plot(temp, label=f"#{loop}")
            ax.set_ylabel(f"{label} {sub_sub_df_cmdPos.loc[idx, label]:.2f}")

for i in range(2):
    axs[1, i].set_xlabel(f"time - time$_0$ [s]")

ax.legend(bbox_to_anchor=(1.01, 1.00))
fig.suptitle(title + "\n")
fig.autofmt_xdate()
fig.tight_layout()

plt.show()

## Data Analysis

Now we know exactly what data we are looking for what are the criteria to select them.  
We want to determine the difference between the commanded position and the actual position once all the Hard-Points are in STANDBY.  
The integration window has an arbitrary length because it would be too hard to guess what would be the next command or event.

We update the `df_results` dataframe to contain the commanded positions.

In [ ]:
for new_col in ["xPositionCmd", "yPositionCmd", "zPositionCmd"]:
    df_results[new_col] = None

Now, we need to extract what were all the commanded positions.  
For this, we the `.groupby` method together with other methods to have a clean and nice DataFrame.

In [ ]:
df_cases = (df_cmdPos
    .groupby(df_cmdPos.columns.to_list())
    .size()
    .reset_index()
    .rename(columns={0:'count'})
    .drop(labels=["count", "xRotation", "yRotation", "zRotation"], axis=1)
)

df_cases = df_cases.rename(columns={col: f"{col}Cmd" for col in df_cases.columns})
df_cases = df_cases.round(2)
df_cases

The cell below walks through each commanded position and fills the `df_results` data frame with them.  

In [ ]:
for idx, case in df_cases.iterrows():
   
    # Select a case
    selected_case = (
        np.isclose(df_cmdPos["xPosition"], case["xPositionCmd"]) &
        np.isclose(df_cmdPos["yPosition"], case["yPositionCmd"]) &
        np.isclose(df_cmdPos["zPosition"], case["zPositionCmd"]) 
    ) 
    temp = df_cmdPos[selected_case]
    
    # Get first STANBY Hard-Point state after sending command to move
    # This means that the Hard-Points are settled down
    for idx_start in temp.index:
        sub_df_HPState = df_HPState[idx_start:]
        mask = (sub_df_HPState[[f"motionState{j}" for j in range(1, 6)]] == 0).all(axis=1) 
        idx_end = sub_df_HPState[mask].index[0]
                
        # Put the commanded positions together with the results
        for col in ["xPositionCmd", "yPositionCmd", "zPositionCmd"]:
            df_results.loc[idx_start:idx_end, col] = case[col]

Now we want to display the data.  
The requirements only need us to check the position and rotation in X and Y only.  
Let's clean our dataframe a bit.  

In [ ]:
df_results = df_results.dropna()
df_results = df_results.drop(columns=["zPosition", "zRotation"])

In [ ]:
df_group = (df_results
    .groupby(["xPositionCmd", "yPositionCmd", "zPositionCmd"])
    .mean()
    .reset_index()
)

df_group

In [ ]:
df_min = (df_results
    .groupby(["xPositionCmd", "yPositionCmd", "zPositionCmd"])
    .min()
    .reset_index()
)

df_min

In [ ]:
df_max = (df_results
    .groupby(["xPositionCmd", "yPositionCmd", "zPositionCmd"])
    .max()
    .reset_index()
)

df_max

In [ ]:
title = f"{test_case} {test_exec}\n Detailed View"
fig, axs = plt.subplots(num=title, figsize=(8, 8), nrows=2, ncols=2)

for i, col in enumerate(["xPosition", "yPosition", "xRotation", "yRotation"]):

    dataset = []
    for idx, row in df_group.iterrows():
        selected_case = (
            (row["xPositionCmd"] == df_results["xPositionCmd"]) & 
            (row["yPositionCmd"] == df_results["yPositionCmd"]) 
        ) 
        dataset.append(df_results.loc[selected_case, col])
    
    ax = axs[i//2, i%2]
    violin = ax.violinplot(dataset)

    for pc in violin["bodies"]:
        pc.set_facecolor(f"C{i}")
        pc.set_alpha(0.25)
    
    for partname in ('cbars', 'cmins', 'cmaxes'):
        vp = violin[partname]
        vp.set_edgecolor(f"C{i}")
        vp.set_linewidth(1)
    
    if "Position" in col:
        ax.set_ylabel(f"{col} Error [mm]")
    else: 
        ax.set_ylabel(f"{col} Error [deg]")
    
    ax.grid(":", alpha=0.1, lw=0.5)

fig.suptitle(title + "\n")
fig.tight_layout()
plt.show()
print("\n\n\n")

In [ ]:
row = 1
title = f"{test_case} {test_exec}\n Detailed and Zoom View - Row {row}"
fig, axs = plt.subplots(num=title, figsize=(8, 8), nrows=2, ncols=2)

for i, col in enumerate(["xPosition", "yPosition", "xRotation", "yRotation"]):

    selected_case = (
        (df_group.iloc[row]["xPositionCmd"] == df_results["xPositionCmd"]) & 
        (df_group.iloc[row]["yPositionCmd"] == df_results["yPositionCmd"]) 
    ) 

    dataset = pd.DataFrame(df_results.loc[selected_case, col])
    
    ax = axs[i//2, i%2]
    ax.plot(dataset, f"C{i}.")
    if "Position" in col:
        ax.set_ylabel(f"{col} Error [mm]")
    else: 
        ax.set_ylabel(f"{col} Error [deg]")
    
    ax.grid(":", alpha=0.1, lw=0.5)

fig.suptitle(title + "\n")
fig.tight_layout()
fig.autofmt_xdate()
plt.show()
print("\n\n\n")